    RetryWithErrorOutputParser: 1. fix the same missformatted_output
                                2. receives the old parser and a model to declare the new parser object
                                3. fix the issue where the OuputFixingParser was not able to. 
                                4. best practice to incorporate these techniques in production is to catch the parsing error using a try: ... except: ... method. 
                                5. capture the errors in the except section and attempt to fix them using the mentioned classes. 
                                6. It will limit the number of API calls and avoid unnecessary costs that are associated with it


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langchain.output_parsers import RetryWithErrorOutputParser

In [2]:
# Define data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [3]:
# Define prompt
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [4]:
model_input = prompt.format_prompt(target_word="behaviour", context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.")


In [5]:
# Define Model
model = OpenAI(model_name='gpt-3.5-turbo-instruct', temperature=0.0)


/Users/srishtysuman/anaconda3/envs/langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [7]:
# parse_with_prompt-> fixes the parsing issue while requiring the output and the prompt.
missformatted_output = '{"words": ["conduct", "manner"]}'

retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)

retry_parser.parse_with_prompt(missformatted_output, model_input)


Suggestions(words=['conduct', 'manner'], reasons=["These words both describe the way in which the students were acting in the classroom, which is similar to the meaning of 'behaviour'."])